In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/23 11:41:17 WARN Utils: Your hostname, DESKTOP-C98VC8U resolves to a loopback address: 127.0.1.1; using 172.23.181.208 instead (on interface eth0)
21/08/23 11:41:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/23 11:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
customers = spark.read.parquet("./output/customers.parquet")
customers.show(3)

+----------------+---------+--------+--------------------+--------+--------------+
|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+----------------+---------+--------+--------------------+--------+--------------+
|               1|    Terri| Leonard|          3124065463|   65369|          Mrs.|
|               2|  Maurice|    West|001-468-075-5301x308|   27061|           Dr.|
|               3|   Taylor|    Hall|    001-004-938-2651|   88302|           Mr.|
+----------------+---------+--------+--------------------+--------+--------------+
only showing top 3 rows



In [4]:
tickets = spark.read.parquet("./output/tickets.parquet")
tickets.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
only showing top 3 rows



In [55]:
tickets.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Net_sales: double (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_season: string (nullable = true)
 |-- Date: date (nullable = true)



In [11]:
#broadcast joining because one table is very small, reduces shuffling
import pyspark.sql.functions as F
join_condition = customers.CustomerIdentity == tickets.customer_id
combined_df =  tickets.join(F.broadcast(customers), join_condition).drop("CustomerIdentity")
combined_df.cache()

DataFrame[ticket_id: int, order_id: int, customer_id: int, quantity: int, Net_sales: double, event_code: string, event_name: string, event_season: string, Date: date, FirstName: string, LASTNAME: string, phone: string, postcode: string, Customer_Title: string]

In [15]:
combined_df.count()

1000

In [16]:
combined_df.select("Date").where(F.col("Date").isNull()).count()

0

In [17]:
combined_df.show(5, truncate=False)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+----------------------+--------+--------------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|event_name        |event_season|Date      |FirstName|LASTNAME|phone                 |postcode|Customer_Title|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+---------+--------+----------------------+--------+--------------+
|1        |182     |160        |1       |58.55    |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Tony     |Baird   |001-097-596-0629x46835|94414   |Ms.           |
|2        |376     |46         |1       |44.25    |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Justin   |Higgins |1889149702            |67579   |Mrs.          |
|3        |282     |116        |5       |47.0     |CHL-ARS   |Chelsea vs Arsenal|2020/2021   |2018-05-05|Jennifer |Dawson  |479.687.6222x409

In [36]:
combined_df.select("customer_id").distinct().count()

198

In [62]:
combined_df.createOrReplaceTempView("DF")

In [68]:
spark.sql("""
          SELECT order_id, Date 
          from DF 
          where Date == "2018-01-30"
          """).show(2)

+--------+----------+
|order_id|      Date|
+--------+----------+
|      20|2018-01-30|
|      14|2018-01-30|
+--------+----------+
only showing top 2 rows



In [70]:
spark.sql("""
          drop database trial
          """)

DataFrame[]

In [52]:
TRANSACTION = combined_df.select("order_id", "Date", "customer_id", "quantity", "Net_sales").distinct().orderBy("Date")
TRANSACTION.count()

1000

In [53]:
TRANSACTION.show(5)

+--------+----------+-----------+--------+---------+
|order_id|      Date|customer_id|quantity|Net_sales|
+--------+----------+-----------+--------+---------+
|     359|2018-01-30|         47|       2|    49.32|
|     182|2018-01-30|        150|       2|    94.19|
|     192|2018-01-30|         69|       5|    84.86|
|      76|2018-01-30|         68|       5|    81.74|
|      98|2018-01-30|         10|       4|    75.13|
+--------+----------+-----------+--------+---------+
only showing top 5 rows



In [54]:
EVENT = combined_df.select("event_name", "event_code", "event_season").distinct().show()

+--------+------------------+----------+------------+
|order_id|        event_name|event_code|event_season|
+--------+------------------+----------+------------+
|      34|       Silverstone|   F1-SILV|   2020/2021|
|     348|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     237|       Silverstone|   F1-SILV|   2020/2021|
|      41|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     205|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     108|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|      83|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|      62|Chelsea vs Arsenal|   CHL-ARS|   2020/2021|
|      13|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     388|       Silverstone|   F1-SILV|   2020/2021|
|      73|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     293|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     200|       Silverstone|   F1-SILV|   2020/2021|
|     227|Chelsea vs Arsenal|   CHL-ARS|   2020/2021|
|     392|  Bath vs Saracens|   BTH-SRC|   2020/2021|
|     378|       Silverstone

21/08/24 08:05:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 48681181 ms exceeds timeout 120000 ms
21/08/24 08:05:39 WARN SparkContext: Killing executors is not supported by current scheduler.


In [39]:
CUSTOMER = combined_df.select("customer_id", "Customer_Title", "FirstName", "LASTNAME", "phone", "postcode").distinct().orderBy("customer_id")
CUSTOMER.show()

+-----------+--------------+-----------+--------+--------------------+--------+
|customer_id|Customer_Title|  FirstName|LASTNAME|               phone|postcode|
+-----------+--------------+-----------+--------+--------------------+--------+
|          1|          Mrs.|      Terri| Leonard|          3124065463|   65369|
|          2|           Dr.|    Maurice|    West|001-468-075-5301x308|   27061|
|          3|           Mr.|     Taylor|    Hall|    001-004-938-2651|   88302|
|          4|           Dr.|Christopher|  Mosley|          9539855170|   23959|
|          5|          Miss|     Lauren|   Davis|       (559)098-0114|   79819|
|          6|           Dr.|     Steven|  Norris|       (160)936-9915|   27807|
|          7|           Mr.|      Randy|    Ross|    267.888.4934x589|   85772|
|          8|           Mr.|   Victoria|Thompson|        534-302-6739|   83797|
|          9|          Mrs.|      Lance|   Moore|          1090466007|   27443|
|         10|           Mr.|     Ashlee|

In [18]:
import pyspark.sql.functions as F
# 3. For each Customer, a list of Events
Edf = combined_df.groupBy("customer_id")\
                 .agg(F.collect_list(F.col("event_code"))\
                 .alias("Events_list"))
Edf.show(5, truncate=False)

+-----------+------------------------------------------------------------------------------------------+
|customer_id|Events_list                                                                               |
+-----------+------------------------------------------------------------------------------------------+
|148        |[BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV, CHL-ARS]                                             |
|31         |[CHL-ARS, CHL-ARS, BTH-SRC, BTH-SRC, CHL-ARS, CHL-ARS, BTH-SRC, CHL-ARS, BTH-SRC, CHL-ARS]|
|85         |[CHL-ARS, F1-SILV, F1-SILV]                                                               |
|137        |[BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV, BTH-SRC, CHL-ARS, BTH-SRC]                           |
|65         |[F1-SILV, BTH-SRC, CHL-ARS]                                                               |
+-----------+------------------------------------------------------------------------------------------+
only showing top 5 rows



In [19]:
# combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()
Tots = combined_df.groupBy("customer_id").agg(F.count("event_name").alias("Totals"))
Tots.show()

+-----------+------+
|customer_id|Totals|
+-----------+------+
|        148|     5|
|         31|    10|
|        137|     7|
|         85|     3|
|         65|     3|
|         53|     1|
|        133|     9|
|         78|     3|
|        108|    11|
|        155|     4|
|         34|     8|
|        193|     6|
|        115|     4|
|        101|     5|
|        126|     1|
|         81|     2|
|        183|     9|
|         28|     1|
|         76|    10|
|         26|     4|
+-----------+------+
only showing top 20 rows



In [26]:
MultiEvent = Tots.withColumn("MultiEvent", F.col("Totals") > 1)
MultiEvent.show()

+-----------+------+----------+
|customer_id|Totals|MultiEvent|
+-----------+------+----------+
|        148|     5|      true|
|         31|    10|      true|
|        137|     7|      true|
|         85|     3|      true|
|         65|     3|      true|
|         53|     1|     false|
|        133|     9|      true|
|         78|     3|      true|
|        108|    11|      true|
|        155|     4|      true|
|         34|     8|      true|
|        193|     6|      true|
|        115|     4|      true|
|        101|     5|      true|
|        126|     1|     false|
|         81|     2|      true|
|        183|     9|      true|
|         28|     1|     false|
|         76|    10|      true|
|         26|     4|      true|
+-----------+------+----------+
only showing top 20 rows



In [28]:
multiEvent1 = lambda x: "true" if x>1 else " "

In [22]:
def multiEvent(x):
    
   if x > 1:

        return   "True"
   else:
        return " "
    

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
multiEvent1_udf = udf(multiEvent1, StringType())

#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
Tots.withColumn("MultiEvent", multiEvent_udf(F.col("Totals"))).show(5)

In [31]:
MultiEvent123 = Tots.withColumn("MultiEvent88", multiEvent1_udf(F.col("Totals")))
MultiEvent123.show()

+-----------+------+------------+
|customer_id|Totals|MultiEvent88|
+-----------+------+------------+
|        148|     5|        true|
|         31|    10|        true|
|        137|     7|        true|
|         85|     3|        true|
|         65|     3|        true|
|         53|     1|            |
|        133|     9|        true|
|         78|     3|        true|
|        108|    11|        true|
|        155|     4|        true|
|         34|     8|        true|
|        193|     6|        true|
|        115|     4|        true|
|        101|     5|        true|
|        126|     1|            |
|         81|     2|        true|
|        183|     9|        true|
|         28|     1|            |
|         76|    10|        true|
|         26|     4|        true|
+-----------+------+------------+
only showing top 20 rows



In [17]:
cols = ["customer_id", "event_name", "event_season", "Date", "rank"]
combined_df.groupBy("customer_id").agg(F.max("quantity")).show()

+-----------+-------------+
|customer_id|max(quantity)|
+-----------+-------------+
|        148|            4|
|         31|            5|
|        137|            5|
|         85|            5|
|         65|            4|
|         53|            2|
|        133|            5|
|         78|            4|
|        108|            5|
|        155|            5|
|         34|            4|
|        193|            5|
|        115|            5|
|        101|            5|
|        126|            5|
|         81|            3|
|        183|            5|
|         28|            3|
|         76|            5|
|         26|            5|
+-----------+-------------+
only showing top 20 rows



In [17]:
# 5. Largest Order by Quantity for each Customer
from pyspark.sql.window import Window
cols = ["customer_id", "quantity"]
windowSpec1 = Window.partitionBy(["customer_id"])\
                    .orderBy(F.desc("quantity"))

combined_df.withColumn("rank", F.rank().over(windowSpec1)).filter("rank == 1").dropDuplicates().select(cols).show()

+-----------+--------+
|customer_id|quantity|
+-----------+--------+
|        148|       4|
|         31|       5|
|         31|       5|
|         31|       5|
|         31|       5|
|         85|       5|
|        137|       5|
|        137|       5|
|         65|       4|
|         53|       2|
|        133|       5|
|        133|       5|
|        133|       5|
|         78|       4|
|         78|       4|
|        108|       5|
|        108|       5|
|        108|       5|
|        155|       5|
|         34|       4|
+-----------+--------+
only showing top 20 rows



In [79]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id").orderBy(F.desc("quantity"))

F = combined_df.withColumn("dddd", F.collect_list(F.col("quantity")).over(WindowSpec)).select("customer_id", "dddd")
F.show()

+-----------+--------------------+
|customer_id|                dddd|
+-----------+--------------------+
|        148|                 [4]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|        [4, 3, 3, 3]|
|        148|     [4, 3, 3, 3, 2]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|        [5, 5, 5, 5]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|  [5, 5, 5, 5, 4, 4]|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         31|[5, 5, 5, 5, 4, 4...|
|         85|                 [5]|
|         85|              [5, 4]|
|         85|           [5, 4, 1]|
|        137|              [5, 5]|
|        137|              [5, 5]|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
F.collect()[2]["dddd"]

In [59]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("customer_id")

df22 = combined_df.withColumn("EventsList", 
                              F.collect_list(F.col("event_code"))
                              .over(WindowSpec)
                              )\
.select("customer_id", "EventsList")
df22.show(5, truncate=False)



+-----------+---------------------------------------------+
|customer_id|EventsList                                   |
+-----------+---------------------------------------------+
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
|148        |[CHL-ARS, BTH-SRC, CHL-ARS, CHL-ARS, F1-SILV]|
+-----------+---------------------------------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="event_code", outputCol="IndexEvents")
indexed = indexer.fit(combined_df).transform(combined_df)
indexed.show(10)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|IndexEvents|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|        0.0|
|        2|     376|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|              46|   Justin| Higgins|          1889149702|   67579|          Mrs.|        0.0|
|        3|     282|       5|     47.0|   CHL-ARS|Chelsea vs Arse

In [18]:
combined_df.groupBy("Customer_Title").agg(F.sum("quantity").alias("Totals")).sort("Totals", ascending=False).show()

+--------------+------+
|Customer_Title|Totals|
+--------------+------+
|           Dr.|  1023|
|           Mr.|   996|
|          Mrs.|   691|
|          Miss|   131|
|           Ms.|   128|
+--------------+------+



In [27]:
add = lambda x,y : x+y
add(2,3)

5

In [22]:
#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
combined_df.groupBy("customer_id").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()

+-----------+------+
|customer_id|Totals|
+-----------+------+
|         53|     1|
|         86|     1|
|        121|     1|
|        126|     1|
|        175|     1|
|         28|     1|
|        149|     1|
|        145|     2|
|         15|     2|
|        156|     2|
|          5|     2|
|         61|     2|
|         24|     2|
|         62|     2|
|          1|     2|
|        117|     2|
|        171|     2|
|         13|     2|
|         81|     2|
|         91|     2|
+-----------+------+
only showing top 20 rows

